<img src="https://raw.githubusercontent.com/dadosaocubo/classificacao/master/DadosAoCubo.png" alt="drawing" width="500"/> 

[__D³__](https://dadosaocubo.com/) by [__Tiago Dias__](https://www.linkedin.com/in/diasctiago/) 

# Bibliotecas

In [ ]:
# Conexão local jupyter notebook
# jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
import requests
import json
#import os
import pandas as pd
#from tqdm import tqdm
#import ast
#from time import sleep, time
import psycopg2
#import sys
#import boto3

# Requisição API

In [ ]:
# Requisição dos dados dos Deputados
url        = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
parametros = {}
resposta   = requests.request("GET", url, params=parametros)
objetos    = json.loads(resposta.text)
dados      = objetos['dados']

In [ ]:
len(dados)

513

In [ ]:
dados[0]

{'id': 204554,
 'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
 'nome': 'Abílio Santana',
 'siglaPartido': 'PL',
 'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
 'siglaUf': 'BA',
 'idLegislatura': 56,
 'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg',
 'email': 'dep.abiliosantana@camara.leg.br'}

In [ ]:
# Requisição dos dados dos Deputados utilizando parametros
url              = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
parametros       = {'id': 204554}
resposta         = requests.request("GET", url, params=parametros)
objetos          = json.loads(resposta.text)
dados_parametros = objetos['dados']

In [ ]:
len(dados_parametros)

1

In [ ]:
dados_parametros[0]

{'id': 204554,
 'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
 'nome': 'Abílio Santana',
 'siglaPartido': 'PL',
 'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
 'siglaUf': 'BA',
 'idLegislatura': 56,
 'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg',
 'email': 'dep.abiliosantana@camara.leg.br'}

# Estruturando dados com Pandas

In [ ]:
df = pd.DataFrame(dados)

In [ ]:
df.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PL,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliosantana@camara.leg.br
1,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abouanni@camara.leg.br
2,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,PROS,https://dadosabertos.camara.leg.br/api/v2/part...,AP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
3,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
4,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,NOVO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adrianaventura@camara.leg.br


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             513 non-null    int64 
 1   uri            513 non-null    object
 2   nome           513 non-null    object
 3   siglaPartido   513 non-null    object
 4   uriPartido     513 non-null    object
 5   siglaUf        513 non-null    object
 6   idLegislatura  513 non-null    int64 
 7   urlFoto        513 non-null    object
 8   email          513 non-null    object
dtypes: int64(2), object(7)
memory usage: 22.1+ KB


In [ ]:
for col in df.columns:
  df[col] = df[col].apply(str)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             513 non-null    object
 1   uri            513 non-null    object
 2   nome           513 non-null    object
 3   siglaPartido   513 non-null    object
 4   uriPartido     513 non-null    object
 5   siglaUf        513 non-null    object
 6   idLegislatura  513 non-null    object
 7   urlFoto        513 non-null    object
 8   email          513 non-null    object
dtypes: object(9)
memory usage: 18.1+ KB


# Persistindo Dados no PostgreSQL

In [ ]:
# Função para criar conexão no banco
def conecta_db():
  con = psycopg2.connect(host='localhost', 
                         database='db_deputados',
                         user='postgres', 
                         password='postgres')
  return con

In [ ]:
# Função para criar tabela no banco
def criar_db(sql):
  con = conecta_db()
  cur = con.cursor()
  cur.execute(sql)
  con.commit()
  con.close()

In [ ]:
# Dropando a tabela caso ela já exista
sql = 'DROP TABLE IF EXISTS public.deputados'
criar_db(sql)
# Criando a tabela dos deputados
sql = '''CREATE TABLE public.deputados 
      ( id            character varying(10), 
        uri           character varying(100), 
        nome          character varying(500), 
        siglaPartido  character varying(50), 
        uriPartido    character varying(200), 
        siglaUf       character varying(10), 
        idLegislatura character varying(10), 
        urlFoto       character varying(100), 
        email         character varying(100) 
      )'''
criar_db(sql)

In [ ]:
# Função para inserir dados no banco
def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()

In [ ]:
# Inserindo cada registro do DataFrame
for i in df.index:
    sql = """
    INSERT into public.deputados (id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email) 
    values('%s','%s','%s','%s','%s','%s','%s','%s','%s');
    """ % (df['id'][i], df['uri'][i], df['nome'][i], df['siglaPartido'][i], df['uriPartido'][i], df['siglaUf'][i], df['idLegislatura'][i], df['urlFoto'][i], df['email'][i])
    inserir_db(sql)

Error: syntax error at or near "Angelo"
LINE 3: ...s.camara.leg.br/api/v2/deputados/141439','Chico D'Angelo','P...
                                                             ^



In [ ]:
# Função para consultas no banco
def consultar_db(sql):
  con = conecta_db()
  cur = con.cursor()
  cur.execute(sql)
  recset = cur.fetchall()
  registros = []
  for rec in recset:
    registros.append(rec)
  con.close()
  return registros

In [ ]:
# Realizando a consulta no PostegreSQL
reg = consulta_db('select * from public.deputados')

In [ ]:
# Tranformando os dados da consulta no PostegreSQL em DataFrame
df_bd = pd.DataFrame(reg, columns=['id','uri','nome','siglaPartido','uriPartido','siglaUf','idLegislatura','urlFoto','email'])
df_bd.head()

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PL,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliosantana@camara.leg.br
1,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abouanni@camara.leg.br
2,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,PROS,https://dadosabertos.camara.leg.br/api/v2/part...,AP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
3,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
4,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,NOVO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adrianaventura@camara.leg.br


In [ ]:
df_bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             512 non-null    object
 1   uri            512 non-null    object
 2   nome           512 non-null    object
 3   siglaPartido   512 non-null    object
 4   uriPartido     512 non-null    object
 5   siglaUf        512 non-null    object
 6   idLegislatura  512 non-null    object
 7   urlFoto        512 non-null    object
 8   email          512 non-null    object
dtypes: object(9)
memory usage: 18.1+ KB
